In [ ]:
# %load init.py
import os
import pickle
import sys
# Enable module import from the parent directory from notebooks
sys.path.append(os.path.abspath('..'))
import time

import matplotlib as mpl
# Select plotting backend
mpl.use('nbAgg')

import matplotlib.pyplot as plt
# Customize plotting
plt.style.use('seaborn-paper')
plt.rcParams['axes.labelsize'] = 11.0
plt.rcParams['axes.titlesize'] = 12.0
plt.rcParams['errorbar.capsize'] = 3.0
plt.rcParams['figure.dpi'] = 72.0
plt.rcParams['figure.titlesize'] = 12.0
plt.rcParams['legend.fontsize'] = 10.
plt.rcParams['lines.linewidth'] = 1.
plt.rcParams['xtick.labelsize'] = 11.0
plt.rcParams['ytick.labelsize'] = 11.0

import numpy as np
import sympy as sp
sp.init_printing(euler=True, use_latex=True)

from IPython import display
from scipy import io, optimize
from sklearn import metrics

import core
import dynamicals
import kernels
import numericals
import utils

In [ ]:
dynamical = dynamicals.LotkaVolterra()

spl_t_0, spl_t_T, spl_freq = 0, 2, 100
obs_t_0, obs_t_T, obs_freq = 0, 2, 10
est_t_0, est_t_T, est_freq = 0, 2, 10
spl_tps, obs_tps, obs_t_indices, est_tps, est_t_indices = utils.create_time(
    spl_t_0, spl_t_T, spl_freq, obs_t_0, obs_t_T, obs_freq, est_t_0, est_t_T, est_freq)
X_0 = np.array([5., 3.]) 
theta = np.array([2., 1., 1., 4.]) 
rho_2 = None
phi = [
    # (Kernal name, Kernal parameters)
    ('rbf', np.sqrt([2.5, 0.02])),
    ('rbf', np.sqrt([2.5, 0.02]))
]
sigma_2 = np.array([0.1, 0.1]) 
delta = np.full(dynamical.num_x, True)
gamma = np.array([5e-3, 5e-3]) 

opt_method = 'Newton-CG'
opt_tol = 1e-6
max_init_iter = None
max_iter = 2000

plotting_enabled = True
plotting_freq = 50
plotting_config = None

In [ ]:
dynamical = dynamicals.ProteinSignallingTransduction() 

spl_t_0, spl_t_T, spl_freq = 0, 100, 20
obs_t_0, obs_t_T, obs_freq = 0, 100, 20
est_t_0, est_t_T, est_freq = 0, 100, 20
spl_tps, obs_tps, obs_t_indices, est_tps, est_t_indices = utils.create_time(
    spl_t_0, spl_t_T, spl_freq, obs_t_0, obs_t_T, obs_freq, est_t_0, est_t_T, est_freq)
t_indices = np.array([0, 1, 2, 4, 5, 7, 10, 15, 20, 30, 40, 50, 60, 80, 100]) * spl_freq
obs_tps = obs_tps[t_indices]
obs_t_indices = obs_t_indices[t_indices]
est_tps = est_tps[t_indices]
est_t_indices = est_t_indices[t_indices]
X_0 = np.array([1., 0., 1., 0., 0.]) 
theta = np.array([0.07, 0.6, 0.05, 0.3, 0.017, 3.]) 
rho_2 = None
sigma_2 = np.full(dynamical.num_x, 1e-2) 
delta = np.full(dynamical.num_x, True) 
# gamma = np.full(dynamical.num_x, 5e-3) 
gamma = np.array([1e-4, 1e-4, 2e-6, 1e-5, 5e-2])
phi = [
    # (Kernal name, Kernal parameters)
    ('sigmoid', np.array([1., .4, 15.])),
    ('sigmoid', np.array([.18, .6, 25.])),
    ('sigmoid', np.array([.84, 3., 3.1])),
    ('sigmoid', np.array([.62, 2., 2.1])),
    ('sigmoid', np.array([.84, 3., 2.9])),
]

opt_method = 'Newton-CG'
opt_tol = 1e-6
max_init_iter = None
max_iter = 2000

plotting_enabled = True
plotting_freq = 50
plotting_config = {
    'x': {
        'xlim': (0, 100),
        'ylim': (0., 1.2)
    }
}

In [ ]:
spl_X = dynamical.generate_sample_path(theta, rho_2, X_0, spl_tps)

In [ ]:
obs_Y = utils.collect_observations(spl_X, obs_t_indices, sigma_2, [0, None])

mu, Sigma, inv_Sigma, m, inv_Lambda, Lambda = core.init_with_gaussian_processes(dynamical, 
                                                                                obs_Y, obs_tps, 
                                                                                est_tps,
                                                                                phi, sigma_2, delta, gamma,
                                                                                plotting_enabled)

utils.plot_states(dynamical, spl_X, spl_tps, obs_Y, obs_tps, delta, mu, est_tps, plotting_config)

In [ ]:
num_x = dynamical.num_x
num_theta = dynamical.num_theta
num_est_t = est_tps.size

X_sym = sp.Matrix(
    list(sp.symbols('{}[(:{})][(:{})]'.format(dynamical.x_label, num_x, num_est_t), real=True))
).reshape(num_x, num_est_t)

theta_sym = sp.Matrix(
    dynamical.theta
)

F_sym = sp.Matrix.hstack(*[
    dynamical.F.xreplace(dict(zip(dynamical.x, X_sym.col(t))))
    for t in range(num_est_t)
])

mx_sym = sp.Matrix.hstack(*[
    sp.Matrix(m[i]) * X_sym.row(i).T
    for i in range(num_x)
]).T

F_sym_mx_sym = F_sym - mx_sym
common_objectives_sym = []
for i in range(num_x):
    print('Constructing common objective for state {}.'.format(i + 1))
    common_objectives_sym.append(
        sp.expand(0.5 * F_sym_mx_sym.row(i) * sp.Matrix(Lambda[i]) * F_sym_mx_sym.row(i).T)
    )
    
theta_gradients_sym = []
for i in range(num_x):
    print('Constructing gradient for theta over state {}'.format(i + 1))
    theta_gradients_sym.append(common_objectives_sym[i].jacobian(theta_sym))

theta_hessians_sym = []
for i in range(num_x):
    print('Constructing hessian for theta over state {}'.format(i + 1))
    theta_hessians_sym.append(theta_gradients_sym[i].jacobian(theta_sym))
    
try:
    print('Constructing common objective functions.')
    common_objectives_func = [
        sp.lambdify((dynamical.dummy_x, dynamical.dummy_theta), sym, dummify=False, modules='numpy')
        for sym in common_objectives_sym
    ]
    
    print('Constructing gradient functions for theta.')
    theta_gradients_sym_func = [
        sp.lambdify((dynamical.dummy_x, dynamical.dummy_theta), sym, dummify=False, modules='numpy')
        for sym in theta_gradients_sym
    ]
    
    print('Constructing hessian functions for theta.')
    theta_hessians_sym_func = [
        sp.lambdify((dynamical.dummy_x, dynamical.dummy_theta), sym, dummify=False, modules='numpy')
        for sym in theta_hessians_sym
    ]
    
    def theta_objective_and_gradient_(theta):
        eta_theta[:] = theta
        objective = 0
        for func in common_objectives_func:
            objective += func(eta_X, theta).ravel()[0]

        gradient = np.zeros(num_theta)
        for func in theta_gradients_sym_func:
            gradient += func(eta_X, theta).ravel()
        return objective, gradient

    def theta_hessian_(theta):
        hessian = np.zeros((num_theta, num_theta))
        for func in theta_hessians_sym_func:
            hessian += func(eta_X, theta)
        return hessian    
except RecursionError as e:
    raise RuntimeError('Unable to construct functions')    

In [ ]:
eta_X = mu.copy()
Xi_X = Sigma.copy()
eta_theta = np.zeros(num_theta)
Xi_theta = np.zeros((num_theta, num_theta))

# Initialization
num_x = dynamical.num_x
num_theta = dynamical.num_theta
num_est_t = est_tps.size

F_funcs = dynamical.F_funcs
x_F_relation = dynamical.x_F_relation
x_dF_funcs = dynamical.x_dF_funcs
x_dF_relation = dynamical.x_dF_relation
x_ddF_funcs = dynamical.x_ddF_funcs
x_ddF_relation = dynamical.x_ddF_relation
# theta_B_funcs = dynamical.theta_B_funcs
# theta_b_funcs = dynamical.theta_b_funcs

mx = np.empty((num_x, num_est_t))  # m.dot(x)
F = np.empty_like(mx)  # F(X, theta)

F_mx = np.empty_like(mx)  # F - mx
Lambda_F_mx = np.empty((num_x, num_est_t))  # Lambda (F - mx)
F_mx_Lambda_F_mx = np.empty(num_x)  # (F - mx)^T Lambda (F - mx)

# theta_B = np.empty((num_x, num_est_t, num_theta))
# theta_b = np.empty((num_x, num_est_t))

# theta_B_Lambda = np.empty((num_x, num_theta, num_est_t))  # theta_B.dot(Lambda)

eta_X = mu.copy()
Xi_X = Sigma.copy()

eta_theta = np.zeros(num_theta)
Xi_theta = np.zeros((num_theta, num_theta))
Xi_theta = np.empty((num_theta, num_theta))
inv_Xi_theta = np.empty_like(Xi_theta)

x_objectives = np.empty(num_x)
x_gradients = np.empty((num_x, num_est_t))
x_hessians = np.empty((num_x, num_est_t, num_est_t))

# Calculate m.dot(x)
for i in range(num_x):
    m[i].dot(eta_X[i], out=mx[i])

# Update theta in closed-form
inv_Xi_theta[:] = 0
for i in range(num_x):
    for n in range(num_theta):
        theta_B[i, :, n] = theta_B_funcs[i][n](eta_X)
    theta_b[i] = theta_b_funcs[i](eta_X)
    theta_B[i].T.dot(Lambda[i], out=theta_B_Lambda[i])
    theta_B_Lambda[i].dot(mx[i] - theta_b[i], out=Psi_theta[i])
    inv_Xi_theta += theta_B_Lambda[i].dot(theta_B[i])
# Xi_theta = numericals.cholesky_inv(inv_Xi_theta)
Xi_theta = np.linalg.pinv(inv_Xi_theta)
Xi_theta.dot(Psi_theta.sum(axis=0), out=eta_theta)
eta_theta = np.abs(eta_theta)

for i in range(num_x):
    F[i] = F_funcs[i](eta_X, eta_theta)
    F_mx[i] = F[i] - mx[i]
    Lambda[i].dot(F_mx[i], out=Lambda_F_mx[i])
    F_mx_Lambda_F_mx[i] = F_mx[i].dot(Lambda_F_mx[i])

def x_objective_and_gradient(x):
    # Necessary update
    eta_X[i] = x
    m[i].dot(eta_X[i], out=mx[i])
    if x_F_relation[i]:
        for j in x_F_relation[i]:
            F[j] = F_funcs[j](eta_X, eta_theta)
            F_mx[j] = F[j] - mx[j]
            Lambda[j].dot(F_mx[j], out=Lambda_F_mx[j])
            F_mx_Lambda_F_mx[j] = F_mx[j].dot(Lambda_F_mx[j])
    else:
        F[i] = F_funcs[i](eta_X, eta_theta)
        F_mx[i] = F[i] - mx[i]
        Lambda[i].dot(F_mx[i], out=Lambda_F_mx[i])
        F_mx_Lambda_F_mx[i] = F_mx[i].dot(Lambda_F_mx[i])

    # Objective
    eta_X_i_mu_i = eta_X[i] - mu[i]
    x_objectives[i] = 0.5 * (eta_X_i_mu_i.dot(inv_Sigma[i].dot(eta_X_i_mu_i)) + F_mx_Lambda_F_mx.sum())

    # Gradient
    inv_Sigma[i].dot(eta_X_i_mu_i, out=x_gradients[i])
    # In numpy Lambda_F_m[i].dot(m[i]) is equivalent to m[i].T.dot(Lambda_F_m[i])
    x_gradients[i] -= Lambda_F_mx[i].dot(m[i])
    for j in x_dF_relation[i]:
        # Element-wise multiplication is used here since the jacobian evaluated is necessarily a diagonal
        # matrix. A diagonal matrix multiplies a vector is equivalent to scaling corresponding elements of
        # the vector by the diagonal entries of the matrix.
        x_gradients[i] += x_dF_funcs[i][j](eta_X, eta_theta) * Lambda_F_mx[j]

    return x_objectives[i], x_gradients[i]

tmp_diagonal = np.zeros((num_est_t, num_est_t))
tmp_diagonal_indices = np.diag_indices_from(tmp_diagonal)

def x_hessian(x):
    x_hessians[i] = inv_Sigma[i].copy()

    for j in x_ddF_relation[i]:
        x_hessians[i][tmp_diagonal_indices] += x_ddF_funcs[i][j](eta_X, eta_theta) * Lambda_F_mx[j]

    if x_dF_relation[i]:
        for j in x_dF_relation[i]:
            tmp = x_dF_funcs[i][j](eta_X, eta_theta)
            if j == i:
                tmp_diagonal[tmp_diagonal_indices] = tmp
                tmp = tmp_diagonal - m[j].T
                x_hessians[i] += tmp.dot(Lambda[j]).dot(tmp.T)
            else:
                if type(tmp) is np.ndarray:
                    x_hessians[i] += tmp.reshape(-1, 1) * Lambda[j] * tmp.reshape(1, -1)
                else:
                    x_hessians[i] += tmp * Lambda[j] * tmp
    else:
        x_hessians[i] += m[i].T.dot(Lambda[i]).dot(m[i])

    return x_hessians[i]

if plotting_enabled is True:
    figure, plotting_config = utils.create_estimation_figure(dynamical, delta)
    utils.add_sample_path(figure, plotting_config, spl_X, spl_tps)
    utils.add_observations(figure, plotting_config, obs_Y, obs_tps, delta)
    utils.add_gaussian_regression_result(figure, plotting_config, eta_X, est_tps)

figure, plotting_config = utils.create_estimation_figure(dynamical, delta)
utils.add_sample_path(figure, plotting_config, spl_X, spl_tps)
utils.add_observations(figure, plotting_config, obs_Y, obs_tps, delta)
utils.add_gaussian_regression_result(figure, plotting_config, eta_X, est_tps)

eta_theta_costs = []
eta_X_costs = []

for it in range(1, 100):
    result = optimize.minimize(fun=theta_objective_and_gradient_, x0=eta_theta, method='Newton-CG', 
                               jac=True, hess=theta_hessian_, options={'disp': False})
    eta_theta = np.abs(result.x)
    eta_theta_cost = result.fun
    eta_theta_costs.append(eta_theta_cost)
    
    eta_X_cost = 0
    for i in range(num_x):
        result = optimize.minimize(fun=x_objective_and_gradient_, x0=eta_X[i], method='Newton-CG', 
                                   jac=True, hess=x_hessian_, options={'disp': False})
        eta_X_cost += result.fun
    eta_X_costs.append(eta_X_cost)
        
    if it % 5 == 0:
        print('Iteration: {}, theta cost: {:.4f}, X cost: {:.4f}'.format(it, eta_theta_cost, eta_X_cost))
        utils.add_estimation_step(figure, plotting_config, dynamical, eta_X, est_tps, theta, eta_theta)

utils.add_estimation_result(figure, plotting_config, dynamical, eta_X, est_tps, theta, eta_theta)
utils.plot_estimation_result(plotting_config, dynamical, spl_X, spl_tps, obs_Y, obs_tps, delta, eta_X, Xi_X,
                             est_tps, theta, eta_theta, Xi_theta)        

In [ ]:
# Helper to plot costs
figure = plt.figure(figsize=plt.figaspect(0.4))
ax = figure.add_subplot(1, 2, 1)
ax.plot(eta_X_costs)
ax.set_title('Cost for X')

ax = figure.add_subplot(1, 2, 2)
ax.plot(eta_X_costs)
ax.set_title('Cost for theta')

figure.tight_layout()
plt.show()

In [ ]:
# Helper to plot theta for protein signaling transduction
figure = plt.figure()
ax = plt.gca()
bar_width = 0.3
bar_indices = np.arange(theta.size - 1)
ax.bar(bar_indices, theta[:-1], bar_width, color='C0', label='Truth')
ax.bar(bar_indices + bar_width, eta_theta[:-1], bar_width, color='C2', label='Estimation',
       yerr=2 * np.sqrt(np.diagonal(Xi_theta))[:-1],
       error_kw=dict(ecolor='0.2', capsize=3., capthick=1.))
ax.set_ylabel('Value')
ax.set_title('Theta')
ax.set_xticks(bar_indices + bar_width / 2)
ax.set_xticklabels([r'${}$'.format(label) for label in dynamical.theta_labels])
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, loc=0)
figure.tight_layout()
plt.show()